In [ ]:
## 수출입 통계를 일별로 변환하기 전에 2018년 이전 데이터는 삭제요청

import pandas as pd
import calendar

# 1️⃣ CSV 불러오기
df = pd.read_csv("../data/양파수출입원본.csv")  # 파일명은 필요 시 수정

# 2️⃣ '합' 행 제거
df = df[df['월'] != '합']

# 3️⃣ 쉼표 제거
cols_to_clean = ['수출(중량)', '수출(금액)', '수입(중량)', '수입(금액)']
for col in cols_to_clean:
    df[col] = df[col].astype(str).str.replace(',', '', regex=True)

# 4️⃣ 숫자형 변환
df['년도'] = df['년도'].astype(float).astype(int)
df['월'] = df['월'].astype(float).astype(int)
for col in cols_to_clean:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 5️⃣ 일별 데이터 생성
rows = []

for _, row in df.iterrows():
    year = int(row['년도'])
    month = int(row['월'])
    last_day = calendar.monthrange(year, month)[1]

    export_weight_per_day = row['수출(중량)'] / last_day
    export_value_per_day  = row['수출(금액)'] / last_day
    import_weight_per_day = row['수입(중량)'] / last_day
    import_value_per_day  = row['수입(금액)'] / last_day

    for day in range(1, last_day + 1):
        date_str = f"{year}-{month:02d}-{day:02d}"
        rows.append({
            '날짜': date_str,
            '수출(중량)': export_weight_per_day,
            '수출(금액)': export_value_per_day,
            '수입(중량)': import_weight_per_day,
            '수입(금액)': import_value_per_day
        })

# 6️⃣ DataFrame 생성
df_daily = pd.DataFrame(rows)

# 7️⃣ CSV 저장
df_daily.to_csv("../data/양파_일별_수출입.csv", index=False, encoding='utf-8-sig')

print("✅ 완료! 파일명: 양파_일별_수출입.csv")


✅ 완료! 파일명: 양파_일별_수출입.csv
